In [1]:
################ convert pdf to images ########################

from pdf2image import convert_from_path
import os

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    for i in range(len(images)):
        images[i].save('C:/matsya/images_mar_24/{}_page{}'.format(filename, i) +'.jpg', 'JPEG')

path = "C:\\matsya\\March-24"

#store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    try:
        pdf_to_images(file, filename)
    except:
         pass

In [2]:
######## remove unwanted files #############

import glob

count = 0
final_images = []
for image in glob.glob('C:/matsya/images_mar_24/*'):
    count += 1
    if 'ms_' in image.lower() or 'mr_' in image.lower():
        continue
    final_images.append(image)

print(count)
print(len(final_images))

164
118


In [3]:
#################### parse output of original images ######################

import glob

for image in final_images:
    filename = os.path.split(image)[-1].split('.jpg')[0]
    psm_values = [4]
    for psm in psm_values:
        parsed_output = 'C:/matsya/output_mar_24/'+filename+'_'+str(psm)
        command = 'tesseract {} {} -l eng --psm {}'.format(image, parsed_output, psm)
        os.system(command)

In [4]:
################### ensemble the output files #####################

import glob
import os

def ensemble_output():

    for file in glob.glob('C:/matsya/output_mar_24/*.txt'):
        filename = os.path.split(file)[-1].split('.txt')[0]
        ind = filename.index('page')
        filename = filename[:ind+5]
        # print(filename)
        with open(file, encoding="utf-8") as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open('C:/matsya/ensemble_files_mar_24/{}.txt'.format(filename), 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")

ensemble_output()

In [45]:
## extract key value pairs

import glob
import os
import datetime
import re
import pandas as pd

def clean_trav_name(traveler_names):
    new_trav = []
    debar = [':', '\'', ' i', '{', '(', ' t', '\\\\', '+', '|', '4', '\\', '!']
    for tn in traveler_names:
        for ch in debar:
            if ch in tn:
                tn = tn.replace(ch, '')
        
        if 'seat' not in tn.lower():
            new_trav.append(tn.strip())
    return new_trav

# def get_traveler_names(text):
#     traveler_names = []
#     flag = 0
#     for i in range(len(text)):
#         if 'pax name' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'airline' in text[i].lower():
#             for j in range(i+1, len(text)-1):
#                 if 'mr' in text[j].lower() or 'ms' in text[j].lower() or 'mrs' in text[j].lower() or 'is' in text[j].lower():
#                     words = text[j].split()
#                     for k in range(len(words)):
#                         words[k] = words[k].replace('|', '')
#                         if ('mr' in words[k].lower() or 'ms' in words[k].lower() or 'mrs' in words[k].lower() or 'is' in words[k].lower()) and len(words[k])<=4:
#                             traveler_name = ''
#                             for l in range(k, len(words)):
#                                 # if (not words[l].isalpha() or words[l].lower() == 'date') and traveler_name and 'ams' not in traveler_name.lower():
#                                 if not words[l].isalpha() and traveler_name and 'ams' not in traveler_name.lower():
#                                     traveler_names.append(traveler_name.strip('‘ '))
#                                     flag = 1
#                                     break
#                                 traveler_name += words[l]+' '
#                             # print('traveler name---> ', traveler_name)
#                 if 'sub-total' in text[j].lower() or 'processing charges' in text[j].lower() or 'terms & conditions' in text[j].lower():
#                     break
#         if flag:
#             break

#     if not traveler_names:
#         traveler_names.append('None')
    
#     new_trav_names = clean_trav_name(traveler_names)
#     return new_trav_names

def get_traveler_names(text):
    traveler_names = []
    traveler_name = ''
    flag = 0
    for i in range(len(text)):
        
        if 'pax name' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'basic fare' in text[i].lower() or ('sector' in text[i].lower() and len(text[i].split())==1):
            for j in range(i+1, len(text)-1):
                if 'sac' in text[j].lower():
                    continue
                row = text[j].split()
                flag = 0
                if 'sub-total' in text[j].lower() or 'processing charges' in text[j].lower():
                    break
                for k in range(2, len(row)):
                    if row[k].isalpha():
                        for l in range(k, len(row)):
                            # print('row[l]--> ', row[l])
                            if '/' in row[l]:
                                # print('traveler name---> ', traveler_name)
                                traveler_names.append(traveler_name.strip())
                                traveler_name = ''
                                flag = 1
                                break
                            traveler_name += row[l] + ' '
                    if flag:
                        break
                   
    # print('traveler names---> ', traveler_names)
    new_trav_names = clean_trav_name(traveler_names)
    # print('new trav names---> ', new_trav_names)
    return new_trav_names


def get_start_dates(text):
    start_dates = []
    start_date = ''
    joinedtext = ' '.join(text)
    n = len(get_traveler_names(text))
    flag = 0
    months = ['jan', 'jgan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'gan', 'aapr', 'gun', 'jgun', 'gul', 'dun', 'jdul', 'dul', '0ct', 'ooct']

    if 'flight details' not in joinedtext.lower() and 'inv.dt.' not in joinedtext.lower():
        
        for line in text:
            if 'crn.dt.' in line.lower():
                start_date = line.split()[-1]
                # start_dates.append(start_date)
                start_dates.extend([start_date]*n)
                # return start_dates

    elif 'inv.dt.' in joinedtext.lower():
        
        for i in range(len(text)):
            if 'inv.dt.' in text[i].lower():
                year = text[i].split()[-1].split('-')[-1]
            if 'pax name' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'airline' in text[i].lower():
                for j in range(i+1, len(text)):
                    words = text[j].split()
                    if ('seat charges' in text[j].lower() or 'meal charges' in text[j].lower() or 'extra baggage' in text[j].lower() or 'excess baggage charges' in text[j].lower()) and 'mr.' not in text[j].lower():
                        continue
                    for word in words:
                        for month in months:
                            if month in word.lower() and '-' in word:
                                flag = 1
                                start_date = word
                                start_date = start_date+'-'+year
                                start_dates.append(start_date)
                                # print('start dates---> ', start_dates)
                                break
                    if 'sub-total' in text[j].lower() or 'processing charges' in text[j].lower() or 'inr' in text[j].lower():
                        break
            if flag:
                break
            # if not flag:
            #     start_dates.append('None')
    
    # print('old start dates---> ', start_dates)
    correct_months = {'31l': '31', 'O1l':'01', 'Jgan': 'Jan', 'Jgun': 'Jun', 'gan': 'Jan', 'Aapr': 'Apr', 'gun': 'Jun', 'aApr': 'Apr', 'gul': 'Jul', 'dun': 'Jun', 'Jdul': 'Jul', 'dul': 'Jul', '0ct': 'Oct', 'Ooct': 'Oct'}
    for i in range(len(start_dates)):
        for k, v in correct_months.items():
            if k in start_dates[i]:
                start_dates[i] = start_dates[i].replace(k, v)
    # print('start dates---> ', start_dates)

    newdates = clean_date(start_dates)

    finaldates = []
    for newdate in newdates:
        finaldate = ''
        formats = ['%d-%b-%Y', '%d-%m-%Y']
        for format in formats:
            try:
                # print('NEWDATE---> ', newdate)
                finaldate = datetime.datetime.strptime(newdate, format).strftime('%Y-%m-%d')
                finaldates.append(finaldate)
            except:
                pass
    
    if not finaldates:
        finaldates.append('None')
        return finaldates
    
    # print('final dates---> ', finaldates)
    return finaldates

def clean_date(dates):
    delete_ch = [':', '_', '_:', '_\'']
    replace_ch = {'O':'0', 'Q':'0', 'l':'1', 'g':'8', 'i':'1', 'S':'5', 'o':'0'}
    newdates = []
    for date in dates:
        newdate = date
        flag = 0
        for c in date:
            if c == '-':
                flag = 1
            for d in delete_ch:
                if d in c:
                    newdate = newdate.replace(d, '')
            if not flag:
                for k, v in replace_ch.items():
                    if k in c:
                        newdate = newdate.replace(k, v)
        newdates.append(newdate)
    
    return newdates

def get_sectors(text):
    sectors = []
    travelers = get_traveler_names(text)
    l = len(travelers)
    flag = 0
    for i in range(len(text)):
        if 'sectors' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'pax name' in text[i].lower() or 'flight details' in text[i].lower() or ('sector' in text[i].lower() and len(text[i].split()) == 1) or 'basic fare' in text[i].lower():
            for j in range(i+1, len(text)):
                words = text[j].split()
                flag = 0
                for word in words:
                    if '/' in word:
                        if len(word.split('/')[0]) > 1 and len(word.split('/')[-1]) > 1:
                            sector = word
                            flag = 1
                            sectors.append(sector)
                            break
                x = re.search(".*sub.*total.*", text[j].lower())
                if x or 'inr' in text[j].lower() or 'terms & conditions' in text[j].lower() or 'processing charges' in text[j].lower():
                    if sectors:
                        # print('sectors---> ', sectors)
                        # if not sectors[0]:
                        #     print('sectors----> ', sectors)
                        return sectors[:l]
    if not flag:
        # if not sectors[0]:
            # print('sectors----> ', sectors)
        sectors.extend(['None']*l)
        # print('sectors none---> ', sectors)
        return sectors 

def divide_sectors(sectors):
    src = []
    dest = []
    for sector in sectors:
        if '/' in sector:
            l = sector.split("/")
        else:
            l = sector.split()
        s = []
        d = []
        for i in range(len(l)-1):
            s.append(l[i].strip())
            d.append(l[i+1].strip())
        src.append(s)
        dest.append(d)
    
    # print('src---> ', src)
    # print('dest---> ', dest)
    return src, dest

def get_city_names(sectors):
    cities = []
    countries = []
    
    for sector in sectors:
        try:
            # print(code)
            value = data[data['iata'] == sector.strip().upper()].iloc[0]
            # print('value---> ', value)
            cities.append(value['city'])
            countries.append(value['country'])
        except Exception as e:
            cities.append('None') 
            countries.append('None')
    
    return cities, countries

def get_invoice(text):
    invoice = []
    inv = ''
    for i in range(len(text)):
        if 'tax & chrg' in text[i].lower() or 'sectors' in text[i].lower() or 'basic fare' in text[i].lower():
            words = text[i+1].split()
            for j in range(len(words)):
                if ('ds' in words[j].lower() or 'ps' in words[j].lower()) and not words[j].isalpha():
                    inv = 'DS_'+words[j+1]
                    break
                elif ('is' in words[j].lower() or '18' in words[j].lower() or '1s' in words[j].lower()) and not words[j].isalpha() and len(words[j]) < 7:
                    inv = 'IS_'+words[j+1]
                    break
        if inv:
            invoice.append(inv)
            return invoice

def get_invoice_date(text):
    for i in range(len(text)):
        if 'crn.dt.' in text[i].lower() or re.search('inv.*.dt.', text[i].lower()):
            words = text[i].split()
            return words[-1]

traveler_names_sheet = []
sectors_sheet = []
start_dates_sheet = []
end_dates_sheet = []
source_cities_sheet = []
dest_cities_sheet = []
source_countries_sheet = []
dest_countries_sheet = []
filename_sheet = []
supplier_sheet = []
invoice_date_sheet = []
invoice_sheet = []

data = pd.read_excel('World_DB.xlsx')

count = 0
for file in glob.glob('C:/matsya/ensemble_files_feb_24/*.txt'):
    # print(file)
    traveler_names_final = []
    start_dates_final = []
    end_dates_final = []
    source_cities_final = []
    dest_cities_final = []
    source_countries_final = []
    dest_countries_final = []
    sectors_final = []
    invoice_date_final = []
    invoice_final = []

    with open(file) as fp:

        filename = os.path.split(file)[-1]
        words = filename.split('_')
        for i in range(len(words)):
            if 'page' in words[i]:
                ind = i
                break
        filename = '_'.join(words[:ind])
        # print(filename)

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            traveler_names = get_traveler_names(text)
            if traveler_names and not traveler_names_final:
                traveler_names_final = traveler_names

            start_dates = get_start_dates(text)
            if start_dates and not start_dates_final:
                start_dates_final = start_dates

            end_dates = get_start_dates(text)
            if end_dates and not end_dates_final:
                end_dates_final = end_dates

            sectors = get_sectors(text)
            if sectors and not sectors_final:
                sectors_final = sectors
            
            # div_sectors = divide_sectors(sectors)
            # print('sectors---> ', sectors)
            source_cities = divide_sectors(sectors)[0]
            if source_cities and not source_cities_final:
                source_cities_final = source_cities

            dest_cities = divide_sectors(sectors)[1]
            if dest_cities and not dest_cities_final:
                dest_cities_final = dest_cities

            inv_date = get_invoice_date(text)
            # print('inv_date---> ', inv_date)
            if inv_date and not invoice_date_final:
                invoice_date_final = inv_date
                # print(invoice_date_final)

            if 'ir' in filename.lower() or 'dr' in filename.lower():
                # count += 1
                # print(filename)
                if 'ir_217' in filename.lower():
                    invoice_final = ['IS_1105']
                if 'ir_406' in filename.lower():
                    invoice_final = ['IS_2039']
                if 'ir_236' in filename.lower():
                    invoice_final = ['IS_1096']
                if 'ir_237' in filename.lower():
                    invoice_final = ['IS_1316']
                if 'ir_240' in filename.lower():
                    invoice_final = ['IS_1325']
                if 'ir_275' in filename.lower():
                    invoice_final = ['IS_1504']
                if 'ir_340' in filename.lower():
                    invoice_final = ['IS_1753']
                if 'ir_385' in filename.lower():
                    invoice_final = ['IS_1889']
                if 'ir_387' in filename.lower():
                    invoice_final = ['IS_1923']
                if 'ir_421' in filename.lower():
                    invoice_final = ['IS_2091']
                invoice = get_invoice(text)
                if invoice and not invoice_final:
                    invoice_final = invoice
            else:
                invoice = ['None']
                if invoice and not invoice_final:
                    invoice_final = invoice
            
            # source_cities = get_source_cities(text)[0]
            # if source_cities and not source_cities_final:
            #     source_cities_final = source_cities

            # dest_cities = get_destination_cities(text)[0]
            # if dest_cities and not dest_cities_final:
            #     dest_cities_final = dest_cities

            # source_countries = get_source_cities(text)[1]
            # if source_countries and not source_countries_final:
            #     source_countries_final = source_countries

            # dest_countries = get_destination_cities(text)[1]
            # if dest_countries and not dest_countries_final:
            #     dest_countries_final = dest_countries

        # if len(traveler_names_final) < len(start_dates_final):
            # traveler_names_final.extend([None]*(len(start_dates_final)-len(traveler_names_final)))
        # traveler_names_sheet.extend(traveler_names_final)
        # sectors_sheet.extend(sectors_final)
        # source_cities_sheet.append(source_cities_final)
        # dest_cities_sheet.append(dest_cities_final)


        # if len(start_dates_final) < len(traveler_names_final):
        #     start_dates_final.extend([None]*(len(traveler_names_final)-len(start_dates_final)))
        # start_dates_sheet.extend(start_dates_final)
        # if len(end_dates_final) < len(traveler_names_final):
        #     end_dates_final.extend([None]*(len(traveler_names_final)-len(end_dates_final)))
        # end_dates_sheet.extend(end_dates_final)
        # if len(source_cities_final) < len(traveler_names_final):
        #     source_cities_final.extend([None]*(len(traveler_names_final)-len(source_cities_final)))
        # source_cities_sheet.extend(source_cities_final)
        # if len(dest_cities_final) < len(traveler_names_final):
        #     dest_cities_final.extend([None]*(len(traveler_names_final)-len(dest_cities_final)))
        # dest_cities_sheet.extend(dest_cities_final)
        # if len(source_countries_final) < len(traveler_names_final):
        #     source_countries_final.extend([None]*(len(traveler_names_final)-len(source_countries_final)))
        # source_countries_sheet.extend(source_countries_final)
        # if len(dest_countries_final) < len(traveler_names_final):
        #     dest_countries_final.extend([None]*(len(traveler_names_final)-len(dest_countries_final)))
        # dest_countries_sheet.extend(dest_countries_final)

        # print(file)
        # print(traveler_names_final)
        # print(source_cities_final)
        # print(dest_cities_final)
        # print(start_dates_final)
        # print(end_dates_final)
        # print()

        # if not source_cities_final[0]:
        #     for i in range(len(source_cities_final)):
        #         source_cities_final[i] = ['None']
        
        # if not dest_cities_final[0]:
        #     for i in range(len(dest_cities_final)):
        #         dest_cities_final[i] = ['None']

        # if not invoice_final:
        #     print('Filename---> ', filename)
        #     print('Traveler Name---> ', traveler_names_final)
        #     print('Start Date---> ', start_dates_final)
        #     print('End Date---> ', end_dates_final)
        #     print('Source City---> ', source_cities_final)
        #     print('Destination City---> ', dest_cities_final)
        #     print('Invoice Final---> ', invoice_final)
        #     print('--------------------------------------------------')

        ## PRINT ALL RECORDS
        
        # if not invoice_date_final:
            # count += 1
        print('Filename---> ', filename)
        print('Traveler Name---> ', traveler_names_final)
        print('Start Date---> ', start_dates_final)
        print('End Date---> ', end_dates_final)
        print('Source City---> ', source_cities_final)
        print('Destination City---> ', dest_cities_final)
        print('Invoice Final---> ', invoice_final)
        print('Invoice Date---> ', invoice_date_final)
        print('--------------------------------------------------')
# print(count)

        # print(traveler_names_final)
        tnames_total = []
        # l=0
        # print(filename)
        for i in range(len(traveler_names_final)):

            if len(source_cities_final) < len(traveler_names_final) or len(start_dates_final) < len(traveler_names_final):
                source_cities_final.append(['None'])
                dest_cities_final.append(['None'])
                start_dates_final.append('None')
                end_dates_final.append('None')
            
            if source_cities_final:
                # print(i)
                # print('traveler names final---> ', traveler_names_final[i])
                # print('source cities final---> ', source_cities_final[i])
                # print('dest cities final---> ', dest_cities_final[i])
                # print('start_dates_final---> ', start_dates_final[i])
                # print('end dates final---> ', end_dates_final[i])
                l = len(source_cities_final[i])
            
            fn = filename.split('_page')[0]
            filename_sheet.extend([fn]*l)
            if traveler_names_final:
                traveler_names_sheet.extend([traveler_names_final[i]]*l)
                tnames_total = [traveler_names_final[i]]*l
            if invoice_final:
                invoice_sheet.extend(invoice_final*len(tnames_total))
            if source_cities_final:
                source_cities_sheet.extend(source_cities_final[i])
            if dest_cities_final:
                dest_cities_sheet.extend(dest_cities_final[i])
            if start_dates_final:
                start_dates_sheet.extend([start_dates_final[i]]*l)
            if end_dates_final:
                end_dates_sheet.extend([end_dates_final[i]]*l)
            if invoice_date_final:
                # print('l---> ', l)
                # print(invoice_date_final)
                invoice_date_sheet.extend([invoice_date_final]*l)
        # print('------------')

        # print('tnames total---> ', tnames_total)
        # count = len(tnames_total)
        # print('count---> ', count)
            
        # invoice_final_each = invoice_final*count
        # if 'dr' in filename.lower() or 'ir' in filename.lower():
        #     print(file)
        #     print(invoice_final_each)
        # invoice_sheet.extend(invoice_final_each)
            
        
        # for i in range(len(start_dates_final)):
        #     l = len(source_cities_final[i])
        #     start_dates_sheet.extend([start_dates_final[i]]*l)
        #     end_dates_sheet.extend([end_dates_final[i]]*l)
    
    # if not source_cities_final[0]:
    #     print(file)
    #     print(traveler_names_final)
    #     print(source_cities_final)
    #     print(dest_cities_final)
    #     print(start_dates_final)
    #     print(end_dates_final)
    #     print()
    
# print(source_cities_sheet)

# filename_sheet.extend([filename]*len(traveler_names_sheet))
supplier_sheet.extend(['MATSYA TRAVEL SERVICES PVT. LTD.']*len(traveler_names_sheet))
if source_cities_sheet:
    source_cities_sheet_final = get_city_names(source_cities_sheet)[0]
if dest_cities_sheet:
    dest_cities_sheet_final = get_city_names(dest_cities_sheet)[0]
if source_cities_sheet:
    source_countries_sheet = get_city_names(source_cities_sheet)[1]
if dest_cities_sheet:
    dest_countries_sheet = get_city_names(dest_cities_sheet)[1]

# print(source_cities_sheet)
# print(source_cities_sheet_final)

print(len(filename_sheet))
print(len(traveler_names_sheet))
print(len(start_dates_sheet))
print(len(end_dates_sheet))
print(len(source_cities_sheet_final))
print(len(dest_cities_sheet_final))
print(len(source_countries_sheet))
print(len(dest_countries_sheet))
print(len(supplier_sheet))
print(len(invoice_date_sheet))
print(len(invoice_sheet))
# print(invoice_date_sheet)

Filename--->  DR_148_page0
Traveler Name--->  ['VISHWA PRATAP']
Start Date--->  ['2024-02-13']
End Date--->  ['2024-02-13']
Source City--->  [['BUR']]
Destination City--->  [['DEL']]
Invoice Final--->  ['DS_1285']
Invoice Date--->  13-02-2024
--------------------------------------------------
Filename--->  DR_156_page0
Traveler Name--->  ['LEONARD HENDRIK THIERRY JAI']
Start Date--->  ['2024-02-15']
End Date--->  ['2024-02-15']
Source City--->  [[]]
Destination City--->  [[]]
Invoice Final--->  ['DS_1312']
Invoice Date--->  15-02-2024
--------------------------------------------------
Filename--->  DS_1265_page0
Traveler Name--->  ['VINEET KAKKAR', 'VINEET KAKKAR']
Start Date--->  ['2024-02-05', '2024-02-10']
End Date--->  ['2024-02-05', '2024-02-10']
Source City--->  [['DEL'], ['MAA']]
Destination City--->  [['MAA'], ['DEL']]
Invoice Final--->  ['None']
Invoice Date--->  29-01-2024
--------------------------------------------------
Filename--->  DS_1266_page0
Traveler Name--->  ['VIBH

In [1]:
import os
import pandas as pd

path = 'C:\matsya\March-24\March-24'

filelist = []
for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))

# print(filelist)

folder = []
file = []
size = []
quality = []

for fl in filelist:
	if 'ms' not in fl.lower() and 'mr' not in fl.lower():
		parts = fl.split('\\')
		folder.append(parts[-2])
		file.append(parts[-1])
		file_size = os.path.getsize(fl)//1024
		size.append(file_size)
		if file_size < 200:
			quality.append('Poor')
		else:
			quality.append('Fine')

# print(folder)
# print(file)
# print(size)
# print(quality)
			
data = {}
data['Folder'] = folder
data['File'] = file
data['File Size'] = size
data['Quality'] = quality

# print(data)

df = pd.DataFrame(data)

In [2]:
df.to_excel('Quality Information.xlsx')

In [47]:
############ create dataframe for all records #################

def df_for_all_records():
    all_invoices = pd.DataFrame({
        'Filename': filename_sheet,
        'Invoice Date': invoice_date_sheet,
        'Traveler Name': traveler_names_sheet,
        'Start Date': start_dates_sheet,
        'End Date': end_dates_sheet,
        'Source City': source_cities_sheet_final,
        'Source Country': source_countries_sheet,
        'Destination City': dest_cities_sheet_final,
        'Destination Country': dest_countries_sheet,
        'Supplier Name': supplier_sheet,
        'Associated Invoice': invoice_sheet
    })

    return all_invoices

In [48]:
############## create dataframe for credit notes ###############

all_invoices = df_for_all_records()
all_invoices.to_excel('feb_24_invoices.xlsx')

def df_for_cred(all_invoices):
    cred_notes = pd.DataFrame()
    cred_notes = all_invoices.loc[(all_invoices['Associated Invoice'] != 'None')]
    return cred_notes

cred_notes = df_for_cred(all_invoices)
cred_notes.to_excel('feb_24_cred_notes.xlsx')

In [2]:
import pandas as pd

all_invoices = pd.read_excel('nov_invoices2.xlsx')

def df_for_cred(all_invoices):
    cred_notes = pd.DataFrame()
    cred_notes = all_invoices.loc[(all_invoices['Associated Invoice'] != 'None')]
    return cred_notes

cred_notes = df_for_cred(all_invoices)
cred_notes.to_excel('nov_cred_notes2.xlsx')

In [104]:
cred_filename = list(cred_notes['Filename'])
# print(len(cred_filename))
ass_inv = list(cred_notes['Associated Invoice'])
# print(len(ass_inv))

removed_invoices = all_invoices[~all_invoices['Filename'].isin(cred_filename)]
removed_invoices = removed_invoices[~removed_invoices['Filename'].isin(ass_inv)]
removed_invoices

removed_invoices.to_excel('feb_24_final_invoices.xlsx')

In [46]:
############# merge both datasets #################

all_invoices = pd.read_excel('C:/sample_invoice_output/matsya/new_all_invoices_total.xlsx')
cred_notes = pd.read_excel('C:/sample_invoice_output/matsya/new_cred_notes_total.xlsx')
# def merge_datasets(all_invoices, cred_notes):
#     merged = pd.merge(all_invoices, cred_notes, on=['Invoice Filename', 'Traveler Name', 'Source City', 'Destination City'])
#     return merged

In [48]:
fns = list(all_invoices['Filename'])

for i in range(len(fns)):
    fns[i] = fns[i].split('_page')[0]

print(fns)

all_invoices['New Filenames'] = fns

['DR_148', 'DR_149', 'DR_150', 'dr_154', 'dr_154', 'dr_154', 'dr_154', 'dr_155', 'DR_160', 'DR_160', 'DR_174', 'DR_174', 'DR_177', 'DR_177', 'DR_178', 'DR_179', 'DR_179', 'DR_179', 'DR_179', 'DR_17', 'DR_17', 'DR_180', 'DR_185', 'DR_187', 'DR_187', 'DR_199', 'DR_207', 'DR_208', 'DR_208', 'DR_209', 'dr_212', 'dr_212', 'DR_28', 'DR_29', 'DR_30', 'DR_31', 'DR_31', 'DR_32', 'DR_32', 'DR_33', 'DR_33', 'DR_34', 'DR_40', 'DR_40', 'DR_41', 'DR_41', 'DR_42', 'DR_43', 'DR_44', 'DR_44', 'DR_45', 'DR_46', 'DR_46', 'DR_47', 'DR_47', 'DR_48', 'DR_48', 'dr_55', 'dr_55', 'dr_56', 'dr_56', 'ds_104', 'ds_104', 'ds_105', 'DS_110', 'DS_110', 'DS_115', 'DS_115', 'DS_11', 'DS_11', 'DS_123', 'DS_12', 'DS_12', 'DS_133', 'DS_133', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1386', 'DS_1386', 'DS_1389', 'DS_1389', 'DS_1390', 'DS_1390', 'DS_1391', 'DS_1391', 'DS_1399', 'DS_1399', 'DS_1400', 'DS_1400', 'DS_1401', 'DS_1401', 'DS_1402', 'DS_1402', 'DS_1403', 'DS_1403'

In [50]:
cred_notes

,SNO,Filename,Traveler Name,Start Date,End Date,Source City,Source Country,Destination City,Destination Country,Supplier Name,Associated Invoice,New Filenames
0,0,DR_148_page0,None,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1211,DR_148
1,1,DR_149_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1305,DR_149
2,2,DR_150_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1354,DR_150
3,3,dr_154_page0,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1383,dr_154
4,4,dr_154_page0,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1383,dr_154
...,...,...,...,...,...,...,...,...,...,...,...,...
94,669,IR_218_page0,MR. ANAND KUMAR SAHAY,2023-09-24,2023-09-24,None,None,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,IS_1211,IR_218
95,670,DR_71_page0,:MR. ANAND KUMAR SAHAY,2023-09-30,2023-09-30,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_711,DR_71
96,671,DR_71_page0,MR. ANAND KUMAR SAHAY,2023-09-30,2023-09-30,Mumbai,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_711,DR_71
97,672,DR_72_page0,MR. PRAVAL SHARMA,2023-09-30,2023-09-30,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_712,DR_72


In [29]:
import pandas as pd

df = pd.DataFrame({
    'Filename': filename_sheet,
    'Traveler Name': traveler_names_sheet,
    'Start Date': start_dates_sheet,
    'End Date': end_dates_sheet,
    'Source City': source_cities_sheet_final,
    'Source Country': source_countries_sheet,
    'Destination City': dest_cities_sheet_final,
    'Destination Country': dest_countries_sheet,
    'Supplier Name': supplier_sheet,
    'Associated Invoice': invoice_sheet
})

df.to_excel('matsya_air_travel3.xlsx')

In [42]:
def pdf_to_images(pdf):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    # for i in range(len(images)):
    images[0].save('C:/sample_invoice_output/imagesample.jpg', 'JPEG')

pdf_to_images('C:/Users/Lenovo/Downloads/Patricia.pdf')

In [44]:
command = 'tesseract C:/sample_invoice_output/imagesample.jpg stdout -l eng --psm 4'
print(os.popen(command).read())
# os.system(command)

nAVAN

You're good to go!

Confirmation details

Hotel confirmation: 83357709

Guest: Jorge Juan Galindo Cruces

You'll need a physical credit card at check-in to pay €834,12

Hotels Summary

Check-in: Sun, May 7, 2023 at 3:00PM
Check-out: Thu, May 11, 2023 at 12:00PM

Hampton By Hilton Utrecht Central Station
Boven Catharijnepoort 4, Utrecht, Utrecht 3511 WN
+31302377033

we & & 3 Star Hotel

9,118 reviews

Your room
Twin Room
1 Twin Bed
Stayokay Hostel %
=) VERDOMHOEKJE de Bijenkorf Utrecht 0 Powech al | &
Koffie & Ik —" _ Broese at
een 9 TivoliVredenbury JANSKERKHOF
Moskee Utrecht Y\  * A Belgisch Biercafé

Olivier Utrecht"
Hoog Catharijne =)

Utrecht
Hott al utrecht) Utrecht Station
8 ? \ELISABETHSTRAAT bigs 3
‘ fa] “MARIAPLAATS so

Google i \ Map data ©2023
Pricing Overview
Number of rooms 1
Duration 4 nights

Price per night 189,05 EUR



In [ ]:
#################### parse output of original images ######################

import glob

for image in final_images:
    filename = os.path.split(image)[-1].split('.jpg')[0]
    psm_values = [3, 4]
    for psm in psm_values:
        parsed_output = 'C:/sample_invoice_output'+filename+'_'+str(psm)
        command = 'tesseract {} {} -l eng --psm {}'.format(image, parsed_output, psm)
        os.system(command)

In [41]:
import glob
import os

def get_invoice(text):
    invoice = []
    inv = ''
    for i in range(len(text)):
        if 'tax & chrg' in text[i].lower():
            words = text[i+1].split()
            for i in range(len(words)):
                if 'ds' in words[i].lower() or 'ps' in words[i].lower():
                    inv = 'DS_'+words[i+1]
                    break
                elif 'is' in words[i].lower() or 'i8' in words[i].lower():
                    inv = 'IS_'+words[i+1]
                    break
        if inv:
            invoice.append(inv)
            return invoice


invoice_sheet = []
count = 0
for file in glob.glob('C:/sample_invoice_output/matsya/ensemble_files/*.txt'):
    invoice_final = []

    with open(file) as fp:

        filename = os.path.split(file)[-1]
        words = filename.split('_')
        for i in range(len(words)):
            if 'page' in words[i]:
                ind = i
                break
        filename = '_'.join(words[:ind])

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts[:1]:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            if 'ir' in filename.lower() or 'dr' in filename.lower():
                count += 1
                print(filename)
                # invoice = get_invoice(text)
            #     if invoice and not invoice_final:
            #         invoice_final = invoice
            # else:
            #     invoice_final = ['None']


        # for i in range(len(traveler_names_final)):
        #     l = len(source_cities_final[i])
        #     invoice_sheet.extend([invoice_final[0]]*l)

    # print(invoice_final)
    # invoice_sheet.extend(invoice_final)

# print(invoice_sheet)
# print(len(invoice_sheet))
print(count)

DR_148_page0
DR_149_page0
DR_150_page0
dr_154_page0
dr_155_page0
DR_160_page0
DR_174_page0
DR_177_page0
DR_178_page0
DR_179_page0
DR_17_page0
DR_180_page0
DR_185_page0
DR_187_page0
DR_199_page0
DR_207_page0
DR_208_page0
DR_209_page0
dr_212_page0
DR_28_page0
DR_29_page0
DR_30_page0
DR_31_page0
DR_32_page0
DR_33_page0
DR_34_page0
DR_40_page0
DR_41_page0
DR_42_page0
DR_43_page0
DR_44_page0
DR_45_page0
DR_46_page0
DR_47_page0
DR_48_page0
dr_55_page0
dr_56_page0
IR_114_page0
IR_189_page0
IR_24_page0
IR_304_page0
IR_305_page0
IR_324_page0
IR_325_page0
IR_326_page0
IR_333_page0
IR_405_page0
IR_406_page0
48


3246

In [43]:
import numpy as np

In [45]:
arr = np.array([[1, 2, 3], [4, 5], [6, 7]])

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_15844/2329706934.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([[1, 2, 3], [4, 5], [6, 7]])


In [46]:
arr

array([list([1, 2, 3]), list([4, 5]), list([6, 7])], dtype=object)

In [47]:
arr.flatten()

array([list([1, 2, 3]), list([4, 5]), list([6, 7])], dtype=object)

In [ ]:
[[1, 2, 3], [4, 5], [6, 7]]

In [2]:
import pandas as pd

all_invoices = pd.read_csv('C:\sample_invoice_output\matsya\matsya-air-travel-Sheet2-csv-xlsx.csv')
credit_notes = pd.read_excel('C:\sample_invoice_output\matsya\credit_note.xlsx')

all_invoices

,SNO,Filename,Traveler Name,Start Date,End Date,Source City,Source Country,Destination City,Destination Country,Supplier Name
0,0,DR_148_page0,MS Shweta,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1,1,DR_149_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.
2,2,DR_150_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.
3,3,dr_154_page0,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.
4,4,dr_154_page0,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.
...,...,...,...,...,...,...,...,...,...,...
1685,1685,IS_1186_page0,MR. RANDHIR KUMAR,2023-09-16,2023-09-16,Abu Dhabi,AE,Riyadh,SA,MATSYA TRAVEL SERVICES PVT. LTD.
1686,1686,IS_1186_page0,MR. RANDHIR KUMAR,2023-09-16,2023-09-16,Riyadh,SA,Abu Dhabi,AE,MATSYA TRAVEL SERVICES PVT. LTD.
1687,1687,IS_1196_page0,MR. AJAY SINGH CHAUHAN,2023-09-18,2023-09-18,Riyadh,SA,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1688,1688,IS_1199_page0,MR. ANAND KUMAR SAHAY,2023-02-24,2023-02-24,Amsterdam,NL,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.


In [3]:
credit_notes

,SNO,Filename,Traveler Name,Start Date,End Date,Source City,Source Country,Destination City,Destination Country,Supplier Name
0,0,DR_148_page0,MS Shweta,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1,1,DR_149_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.
2,2,DR_150_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.
3,3,dr_154_page0,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.
4,4,dr_154_page0,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.
...,...,...,...,...,...,...,...,...,...,...
89,632,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Ho Chi Minh City,VN,Singapore,SG,MATSYA TRAVEL SERVICES PVT. LTD.
90,633,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Singapore,SG,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.
91,1655,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,New Delhi,IN,Amsterdam,NL,MATSYA TRAVEL SERVICES PVT. LTD.
92,1656,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,Amsterdam,NL,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.


In [37]:
all_invoices['Start Date'] = pd.to_datetime(all_invoices['Start Date'])

In [39]:
merged = pd.merge(all_invoices, credit_notes, on=['Source City', 'Destination City', 'Traveler Name', 'Start Date'])
merged.to_excel('merged4.xlsx')

In [41]:
sno_merged = list(merged['SNO_x'])
print(len(sno_merged))

105


In [20]:
merged

,SNO_x,Filename,Traveler Name,Start Date_x,End Date_x,Source City,Source Country_x,Destination City,Destination Country_x,Supplier Name_x,SNO_y,Start Date_y,End Date_y,Source Country_y,Destination Country_y,Supplier Name_y
0,0,DR_148_page0,MS Shweta,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.,0,2022-12-27,2022-12-27,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1,1,DR_149_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.,1,2022-12-27,2022-12-27,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
2,2,DR_150_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,2,2022-12-27,2022-12-27,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
3,3,dr_154_page0,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,3,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
4,4,dr_154_page0,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,4,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,632,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Ho Chi Minh City,VN,Singapore,SG,MATSYA TRAVEL SERVICES PVT. LTD.,632,2023-03-23,2023-03-23,VN,SG,MATSYA TRAVEL SERVICES PVT. LTD.
90,633,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Singapore,SG,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,633,2023-03-23,2023-03-23,SG,IN,MATSYA TRAVEL SERVICES PVT. LTD.
91,1655,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,New Delhi,IN,Amsterdam,NL,MATSYA TRAVEL SERVICES PVT. LTD.,1655,2023-09-20,2023-09-20,IN,NL,MATSYA TRAVEL SERVICES PVT. LTD.
92,1656,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,Amsterdam,NL,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,1656,2023-09-20,2023-09-20,NL,IN,MATSYA TRAVEL SERVICES PVT. LTD.


In [43]:
removed_invoices = all_invoices[~all_invoices['SNO'].isin(sno_merged)]
removed_invoices

removed_invoices.to_excel('final_invoices.xlsx')

1585

In [54]:
def get_invoice(text):
    invoice = []
    inv = ''
    for i in range(len(text)):
        if 'tax & chrg' in text[i].lower():
            words = text[i+1].split()
            for j in range(len(words)):
                if ('ds' in words[j].lower() or 'ps' in words[j].lower()) and not words[j].isalpha():
                    inv = 'DS_'+words[j+1]
                    break
                elif ('is' in words[j].lower() or '18' in words[j].lower()) and not words[j].isalpha() and len(words[j]) < 7:
                    inv = 'IS_'+words[j+1]
                    break
        if inv:
            # invoice.append(inv)
            # return invoice
            return inv

In [47]:
filenames = list(credit_notes['Filename'])
filenames

['DR_148_page0',
 'DR_149_page0',
 'DR_150_page0',
 'dr_154_page0',
 'dr_154_page0',
 'dr_154_page0',
 'dr_154_page0',
 'dr_155_page0',
 'DR_160_page0',
 'DR_160_page0',
 'DR_174_page0',
 'DR_174_page0',
 'DR_177_page0',
 'DR_177_page0',
 'DR_178_page0',
 'DR_179_page0',
 'DR_179_page0',
 'DR_179_page0',
 'DR_179_page0',
 'DR_17_page0',
 'DR_17_page0',
 'DR_180_page0',
 'DR_185_page0',
 'DR_187_page0',
 'DR_187_page0',
 'DR_199_page0',
 'DR_207_page0',
 'DR_208_page0',
 'DR_208_page0',
 'DR_209_page0',
 'dr_212_page0',
 'dr_212_page0',
 'DR_28_page0',
 'DR_29_page0',
 'DR_30_page0',
 'DR_31_page0',
 'DR_31_page0',
 'DR_32_page0',
 'DR_32_page0',
 'DR_33_page0',
 'DR_33_page0',
 'DR_34_page0',
 'DR_40_page0',
 'DR_40_page0',
 'DR_41_page0',
 'DR_41_page0',
 'DR_42_page0',
 'DR_43_page0',
 'DR_44_page0',
 'DR_44_page0',
 'DR_45_page0',
 'DR_46_page0',
 'DR_46_page0',
 'DR_47_page0',
 'DR_47_page0',
 'DR_48_page0',
 'DR_48_page0',
 'dr_55_page0',
 'dr_55_page0',
 'dr_56_page0',
 'dr_56_pa

In [ ]:
import glob
import os

for file in glob.glob('C:/sample_invoice_output/matsya/ensemble_files/*.txt'):
    invoice_final = []

    with open(file) as fp:

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts[:1]:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]

            filename = os.path.split(file)[-1].split('.')[0]
            # words = filename.split('_')
            # for i in range(len(words)):
            #     if 'page' in words[i]:
            #         ind = i
            #         break
            # filename = '_'.join(words[:ind])
            # print(filename)
            if filename in filenames:
                inv = get_invoice(text)
                print(str(inv))

In [ ]:
count = 0
for fname in filenames:
    for file in glob.glob('C:/sample_invoice_output/matsya/ensemble_files/*.txt'):

        with open(file) as fp:

            texts = fp.read().split("---\n")
            # pprint(texts)
            for text in texts[:1]:
                text = text.splitlines()
                text = [line for line in text if not line.isspace() and len(line) > 0]

                filename = os.path.split(file)[-1].split('.')[0]
                # words = filename.split('_')
                # for i in range(len(words)):
                #     if 'page' in words[i]:
                #         ind = i
                #         break
                # filename = '_'.join(words[:ind])
                # print(filename)
                if filename in fname:
                    count += 1
                    inv = get_invoice(text)
                    print(str(inv))
# print(count)

In [34]:
cred_with_invoice = pd.read_excel('C:\sample_invoice_output\matsya\matsya_air_travel_final.xlsx')
cred_with_invoice

,SNO,Filename,Traveler Name,Start Date,End Date,Source City,Source Country,Destination City,Destination Country,Supplier Name,Invoice Filename
0,0,DR_148_page0,MS Shweta,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1211
1,1,DR_149_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1305
2,2,DR_150_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1354
3,3,dr_154_page0,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1383
4,4,dr_154_page0,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1383
...,...,...,...,...,...,...,...,...,...,...,...
89,632,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Ho Chi Minh City,VN,Singapore,SG,MATSYA TRAVEL SERVICES PVT. LTD.,IS_2039
90,633,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Singapore,SG,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,IS_2039
91,1655,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,New Delhi,IN,Amsterdam,NL,MATSYA TRAVEL SERVICES PVT. LTD.,IS_1105
92,1656,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,Amsterdam,NL,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,IS_1105


In [35]:
all_renamed_invoices = pd.read_excel('C:/sample_invoice_output/matsya/all_renamed_invoices.xlsx')
all_renamed_invoices

,SNO,Filename,Traveler Name,Start Date,End Date,Source City,Source Country,Destination City,Destination Country,Supplier Name
0,0,DR_148,MS Shweta,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1,1,DR_149,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.
2,2,DR_150,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.
3,3,dr_154,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.
4,4,dr_154,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.
...,...,...,...,...,...,...,...,...,...,...
1685,1685,IS_1186,MR. RANDHIR KUMAR,2023-09-16,2023-09-16,Abu Dhabi,AE,Riyadh,SA,MATSYA TRAVEL SERVICES PVT. LTD.
1686,1686,IS_1186,MR. RANDHIR KUMAR,2023-09-16,2023-09-16,Riyadh,SA,Abu Dhabi,AE,MATSYA TRAVEL SERVICES PVT. LTD.
1687,1687,IS_1196,MR. AJAY SINGH CHAUHAN,2023-09-18,2023-09-18,Riyadh,SA,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1688,1688,IS_1199,MR. ANAND KUMAR SAHAY,2023-02-24,2023-02-24,Amsterdam,NL,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.


In [36]:
all_renamed_invoices.rename(columns={'Filename': 'Invoice Filename'}, inplace=True)

In [37]:
merged = pd.merge(all_renamed_invoices, cred_with_invoice, on=['Source City', 'Destination City', 'Traveler Name', 'Invoice Filename'])
merged.to_excel('merged5.xlsx')

In [38]:
merged

,SNO_x,Invoice Filename,Traveler Name,Start Date_x,End Date_x,Source City,Source Country_x,Destination City,Destination Country_x,Supplier Name_x,SNO_y,Filename,Start Date_y,End Date_y,Source Country_y,Destination Country_y,Supplier Name_y
0,75,DS_1383,MR. ANAND KUMAR SAHAY,2022-01-03,2022-01-03,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,3,dr_154_page0,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
1,79,DS_1383,MR. ANAND KUMAR SAHAY,2022-01-03,2022-01-03,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,3,dr_154_page0,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
2,76,DS_1383,MR. PRAVAL SHARMA,2022-01-03,2022-01-03,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,4,dr_154_page0,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
3,80,DS_1383,MR. PRAVAL SHARMA,2022-01-03,2022-01-03,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,4,dr_154_page0,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
4,77,DS_1383,MR. ANAND KUMAR SAHAY,2022-01-03,2022-01-03,Mumbai,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,5,dr_154_page0,2023-01-10,2023-01-10,IN,IN,MATSYA TRAVEL SERVICES PVT. LTD.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1619,IS_9,MR. ANAND KUMAR SAHAY,2023-04-10,2023-04-10,New Delhi,IN,Zurich,CH,MATSYA TRAVEL SERVICES PVT. LTD.,610,IR_24_page0,2023-04-10,2023-04-10,IN,CH,MATSYA TRAVEL SERVICES PVT. LTD.
79,1620,IS_9,MR. ANAND KUMAR SAHAY,2023-04-10,2023-04-10,Zurich,CH,Amsterdam,NL,MATSYA TRAVEL SERVICES PVT. LTD.,611,IR_24_page0,2023-04-10,2023-04-10,CH,NL,MATSYA TRAVEL SERVICES PVT. LTD.
80,1621,IS_9,MR. ANAND KUMAR SAHAY,2023-04-10,2023-04-10,Amsterdam,NL,Zurich,CH,MATSYA TRAVEL SERVICES PVT. LTD.,612,IR_24_page0,2023-04-10,2023-04-10,NL,CH,MATSYA TRAVEL SERVICES PVT. LTD.
81,1622,IS_9,MR. ANAND KUMAR SAHAY,2023-04-10,2023-04-10,Zurich,CH,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,613,IR_24_page0,2023-04-10,2023-04-10,CH,IN,MATSYA TRAVEL SERVICES PVT. LTD.


In [39]:
sno_all = list(merged['SNO_x'])
sno_all.extend(cred_with_invoice['SNO'])
print(sno_all)

[75, 79, 76, 80, 77, 81, 78, 82, 186, 199, 203, 204, 223, 224, 227, 228, 234, 239, 241, 243, 245, 258, 275, 276, 305, 308, 309, 312, 337, 338, 413, 414, 415, 416, 417, 424, 425, 489, 495, 512, 513, 516, 517, 518, 519, 520, 521, 522, 543, 550, 551, 700, 701, 730, 731, 748, 749, 750, 751, 780, 781, 782, 783, 882, 883, 884, 885, 945, 1107, 1108, 1176, 1177, 1449, 1450, 1451, 1452, 1554, 1555, 1619, 1620, 1621, 1622, 1689, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 1655, 1656, 1657]


In [76]:
removed_invoices = all_renamed_invoices[all_invoices['SNO'].isin(sno_all)]
removed_invoices

removed_invoices.to_excel('removed_invoices.xlsx')

In [77]:
1690-1513

177

In [78]:
cred_notes_invoices = pd.read_excel('C:\sample_invoice_output\matsya\credit_notes.xlsx')

In [85]:
cred_notes_invoices

,SNO,Filename,Traveler Name,Start Date,End Date,Source City,Source Country,Destination City,Destination Country,Supplier Name,Invoice Filename
0,0,DR_148_page0,MS Shweta,2022-12-27,2022-12-27,Jaipur,IN,Pune,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1211
1,1,DR_149_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,New Delhi,IN,Bangalore,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1305
2,2,DR_150_page0,MR. ROHAN SRIVASTAVA,2022-12-27,2022-12-27,Bangalore,IN,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1354
3,3,dr_154_page0,MR. ANAND KUMAR SAHAY,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1383
4,4,dr_154_page0,MR. PRAVAL SHARMA,2023-01-10,2023-01-10,New Delhi,IN,Mumbai,IN,MATSYA TRAVEL SERVICES PVT. LTD.,DS_1383
...,...,...,...,...,...,...,...,...,...,...,...
89,632,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Ho Chi Minh City,VN,Singapore,SG,MATSYA TRAVEL SERVICES PVT. LTD.,IS_2039
90,633,IR_406_page0,MR. ASHISH PROOTHI,2023-03-23,2023-03-23,Singapore,SG,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,IS_2039
91,1655,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,New Delhi,IN,Amsterdam,NL,MATSYA TRAVEL SERVICES PVT. LTD.,IS_1105
92,1656,IR_217_page0,MS SWARNA PRADHAN PRAHA,2023-09-20,2023-09-20,Amsterdam,NL,New Delhi,IN,MATSYA TRAVEL SERVICES PVT. LTD.,IS_1105


In [ ]:
total_invoices = pd.read_excel('C:\sample_invoice_output\matsya/total_invoices.xlsx')
# total_invoices = total_invoices.values.tolist()
# print(total_invoices)
total_invoices

In [179]:
fns = list(total_invoices['Filename'])

for i in range(len(fns)):
    fns[i] = fns[i].split('_page')[0]

print(fns)

total_invoices['New Filenames'] = fns

['DR_148', 'DR_149', 'DR_150', 'dr_154', 'dr_154', 'dr_154', 'dr_154', 'dr_155', 'DR_160', 'DR_160', 'DR_174', 'DR_174', 'DR_177', 'DR_177', 'DR_178', 'DR_179', 'DR_179', 'DR_179', 'DR_179', 'DR_17', 'DR_17', 'DR_180', 'DR_185', 'DR_187', 'DR_187', 'DR_199', 'DR_207', 'DR_208', 'DR_208', 'DR_209', 'dr_212', 'dr_212', 'DR_28', 'DR_29', 'DR_30', 'DR_31', 'DR_31', 'DR_32', 'DR_32', 'DR_33', 'DR_33', 'DR_34', 'DR_40', 'DR_40', 'DR_41', 'DR_41', 'DR_42', 'DR_43', 'DR_44', 'DR_44', 'DR_45', 'DR_46', 'DR_46', 'DR_47', 'DR_47', 'DR_48', 'DR_48', 'dr_55', 'dr_55', 'dr_56', 'dr_56', 'ds_104', 'ds_104', 'ds_105', 'DS_110', 'DS_110', 'DS_115', 'DS_115', 'DS_11', 'DS_11', 'DS_123', 'DS_12', 'DS_12', 'DS_133', 'DS_133', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1383', 'DS_1386', 'DS_1386', 'DS_1389', 'DS_1389', 'DS_1390', 'DS_1390', 'DS_1391', 'DS_1391', 'DS_1399', 'DS_1399', 'DS_1400', 'DS_1400', 'DS_1401', 'DS_1401', 'DS_1402', 'DS_1402', 'DS_1403', 'DS_1403'

In [184]:
modified_df = pd.DataFrame()
index1 = 0
for index, row in cred_notes_invoices.iterrows():
   modified_df = modified_df.append(row, ignore_index=True)
   filename = row['Invoice Filename']
   traveler_name = row['Traveler Name']
   source_city = row['Source City']
   dest_city = row['Destination City']
   # print(filename, traveler_name, source_city, dest_city)
   # print(total_invoices['Filename'][index])

   # total_filenames = total_invoices['Filename'][index].split('_page')[0]

   modified_df = modified_df.append(total_invoices[(total_invoices['New Filenames'] == filename) & (total_invoices['Traveler Name'] == traveler_name) & (total_invoices['Source City'] == source_city) & (total_invoices['Destination City'] == dest_city)])
   # print('yes')
   #    index1 += 1

modified_df.to_excel('modified_df.xlsx')
   

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_13352/209591878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modified_df = modified_df.append(row, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_13352/209591878.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modified_df = modified_df.append(total_invoices[(total_invoices['New Filenames'] == filename) & (total_invoices['Traveler Name'] == traveler_name) & (total_invoices['Source City'] == source_city) & (total_invoices['Destination City'] == dest_city)])
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_13352/209591878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modified_df = modified_df.append(row, ignore_index=True)
C:\Users\Lenovo\AppData\Local

In [181]:
total_invoices[(total_invoices['New Filenames'] == filename) & (total_invoices['Traveler Name'] == traveler_name) & (total_invoices['Source City'] == source_city) & (total_invoices['Destination City'] == dest_city)]

0       False
1       False
2       False
3       False
4       False
        ...  
1685    False
1686    False
1687    False
1688    False
1689     True
Length: 1690, dtype: bool

In [135]:
cred_notes_invoices['Invoice Filename'][index]

'DS_1211'

In [168]:
total_invoices["Traveler Name"] == cred_notes_invoices['Traveler Name'][index]

0        True
1       False
2       False
3       False
4       False
        ...  
1685    False
1686    False
1687    False
1688    False
1689    False
Name: Traveler Name, Length: 1690, dtype: bool

In [ ]:
#################### parse output of original images ######################

import glob
import os

for image in glob.glob(path):
    filename = os.path.split(image)[-1].split('.jpg')[0]
    psm_values = [3, 4]
    langs = ['eng', 'eng+hin']
    for psm in psm_values:
        for lang in langs:
            parsed_output = 'C:/sample_invoice_output/matsya/output2/'+filename+'_'+lang+'_'+str(psm)
            command = 'tesseract {} {} -l {} --psm {}'.format(image, parsed_output, lang, psm)
            os.system(command)

In [2]:
import pandas as pd

final_invoices = pd.read_excel('C:\sample_invoice_output\matsya\matsya_at_final_invoices.xlsx')
main_sheet_xebia = pd.read_excel('C:\sample_invoice_output\matsya\main_sheet_xebia.xlsx')

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_14912/4286217140.py:3: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  final_invoices = pd.read_excel('C:\sample_invoice_output\matsya\matsya_at_final_invoices.xlsx')


In [10]:
final_invoices.columns

Index(['SNO', 'Invoice Filename', 'Traveler Name', 'Start Date', 'End Date',
       'Source City', 'Source Country', 'Destination City',
       'Destination Country', 'Supplier Name', 'Unnamed: 10'],
      dtype='object')

In [11]:
main_sheet_xebia.columns

Index(['Asset Name', 'Traveler Name', 'Start Date', 'End Date', 'Distance',
       'Distance Unit', 'Source City', 'Source Country', 'Destination City',
       'Destination Country', 'Supplier Name', 'Filename'],
      dtype='object')

In [8]:
# final_invoices.columns
# main_sheet_xebia.columns
main_sheet_xebia['Filename'] = '' 

In [17]:
new_df = pd.DataFrame()

Index(['Asset Name', 'Traveler Name', 'Start Date', 'End Date', 'Distance',
       'Distance Unit', 'Source City', 'Source Country', 'Destination City',
       'Destination Country', 'Supplier Name', 'Filename'],
      dtype='object')

In [ ]:
i = 0
for index in range(len(main_sheet_xebia)):
    row1 = final_invoices.iloc[index]
    row2 = main_sheet_xebia.iloc[index]
    if (row1['Traveler Name'] == row2['Traveler Name']) & (row1['Start Date'] == row2['Start Date']) & (row1['End Date'] == row2['End Date']) & (row1['Source City'] == row2['Source City']) & (row1['Source Country'] == row2['Source Country']) & (row1['Destination City'] == row2['Destination City']) & (row1['Destination Country'] == row2['Destination Country']):
        print('row1---> ', row1)
        print('row2---> ', row2)
        print('------')
        # new_df[i] = row1
        # new_df[i+1] = row2
        # i += 1
# print(new_df)


In [30]:
duplicates = final_invoices[final_invoices.duplicated(['Traveler Name', 'Start Date', 'End Date', 'Source City', 'Destination City'], keep=False)]
# print(duplicates)
duplicates.to_excel('duplicates.xlsx')

In [28]:
duplicates['Filename'] = ''


C:\Users\Lenovo\AppData\Local\Temp/ipykernel_14912/38445790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates['Filename'] = ''


In [29]:
for index in range(len(duplicates)):
    row1 = final_invoices.iloc[index]
    row2 = duplicates.iloc[index]
    if (row1['Traveler Name'] == row2['Traveler Name']) & (row1['Start Date'] == row2['Start Date']) & (row1['End Date'] == row2['End Date']) & (row1['Source City'] == row2['Source City']) & (row1['Source Country'] == row2['Source Country']) & (row1['Destination City'] == row2['Destination City']) & (row1['Destination Country'] == row2['Destination Country']):
        duplicates['Filename'] = row1['Filename']
    else:
        

Index(['Asset Name', 'Traveler Name', 'Start Date', 'End Date', 'Distance',
       'Distance Unit', 'Source City', 'Source Country', 'Destination City',
       'Destination Country', 'Supplier Name', 'Filename'],
      dtype='object')

In [32]:
template

,Asset Name,Traveler Name,Start Date,End Date,Distance,Distance Unit,Source City,Source Country,Destination City,Destination Country,Supplier Name,Supplemental Emission


In [ ]:
import pandas as pd

template = pd.read_excel('C:\sample_invoice_output\matsya\Business Travel Genpact Data.xlsx')
data = pd.read_excel('C:\sample_invoice_output\matsya\Category 6- Business Travel.xlsx')

def get_dates(month):
    months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    odd = ['Jan', 'Mar', 'May', 'Jul', 'Aug', 'Oct', 'Dec']
    even = ['Apr', 'Jun', 'Sep', 'Nov']
    m = month.split('\'')[0]
    y = month.split('\'')[1]
    start_date = '01-'+months[m]+'-20'+y
    if m in odd:
        end_date = '31-'+months[m]+'-20'+y
    elif m in even:
        end_date = '30-'+months[m]+'-20'+y
    else:
        end_date = '28-'+months[m]+'-20'+y
    return start_date, end_date

sup_em = 12
for j in range(2, 8):
    num = 1
    for i in range(3, 15):
        start_date, end_date = get_dates(data.iloc[1][i])
        row = {}
        row = {'Asset Name': 'None', 'Traveler Name': 'Traveler'+str(num), 'Start Date': start_date, 'End Date': end_date, 'Distance': round(data.iloc[j][i]), 'Distance Unit': data.iloc[j][2], 'Source City': 'None', 'Source Country': 'None', 'Destination City': 'None', 'Destination Country': 'None', 'Supplier Name': 'None', 'Supplemental Emission': round(data.iloc[sup_em][i])}
        template = template.append(row, ignore_index=True)
        num += 1
    sup_em += 1

# print(template)
template.to_excel('template.xlsx')


In [127]:
## 9-141

water_303_1 = pd.read_excel('C:\sample_invoice_output\matsya/303-1Water.xlsx')
stn_asset_water_act = pd.read_csv('C:\sample_invoice_output\matsya\stnryAssetWaterActvty.csv')

def get_dates(month):
    months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'June': '06', 'July': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    odd = ['Jan', 'Mar', 'May', 'July', 'Aug', 'Oct', 'Dec']
    even = ['Apr', 'June', 'Sep', 'Nov']
    # print('month---> ', month)
    month = month.replace('\'', '')
    m = month.split()[0]
    y = month.split()[1]
    start_date = '01-'+months[m]+'-20'+y
    if m in odd:
        end_date = '31-'+months[m]+'-20'+y
    elif m in even:
        end_date = '30-'+months[m]+'-20'+y
    else:
        end_date = '28-'+months[m]+'-20'+y
    return start_date, end_date

sup_em = 12
mapping = {'Borewell': 'GrndWaterRenewable', 'Municipal Water Supply- Tanker/connection': 'ThirdPartyReserve', 'Private Water Supply-Tanker': 'ThirdPartyReserve', 'Packaged Drinking water': 'FreshWater', 'Any Other Source': 'RecycledWater', 'Any Other Source (RO)': 'FreshWater'}
for j in range(7, 139):
    
    for i in range(4, 16):
        start_date, end_date = get_dates(water_303_1.iloc[6][i])
        asset_name = water_303_1.iloc[j][0]
        act_src_type = water_303_1.iloc[j][2]
        # print('act src type---> ', act_src_type)
        quantity = water_303_1.iloc[j][i]
        row = {}
        # print(act_src_type)
        if act_src_type.strip() in mapping:
            row = {'Asset Name': asset_name, 'Service Provider(Supplier)': 'TestSup', 'Start Date': start_date, 'End Date': end_date, 'Activity Type': 'Withdrawal', 'Activity Source Type': mapping[act_src_type.strip()], 'Treatment Type': 'None', 'Quantity': quantity, 'Quantity Unit': 'Liters'}
            stn_asset_water_act = stn_asset_water_act.append(row, ignore_index=True)
        
    sup_em += 1
    # break

# print(stn_asset_water_act)

#stn_asset_water_act.to_excel('stn_asset_water_act.xlsx')

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4136/2149687360.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stn_asset_water_act = stn_asset_water_act.append(row, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4136/2149687360.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stn_asset_water_act = stn_asset_water_act.append(row, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4136/2149687360.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stn_asset_water_act = stn_asset_water_act.append(row, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_4136/2149687360.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [96]:
stn_asset_water_act['Activity Source Type'].apply(lambda x: d.get(x))

0                     None
1       GrndWaterRenewable
2       GrndWaterRenewable
3       GrndWaterRenewable
4       GrndWaterRenewable
               ...        
1292                  None
1293                  None
1294                  None
1295                  None
1296                  None
Name: Activity Source Type, Length: 1297, dtype: object

In [130]:
stn_asset_water_act['Quantity'] = stn_asset_water_act['Quantity'].replace(0, np.nan)

In [141]:
stn_asset_water_act.isna().sum()

Asset Name                      0
Service Provider(Supplier)      0
Start Date                      0
End Date                        0
Activity Type                   0
Activity Source Type            0
Treatment Type                  0
Quantity                      965
Quantity Unit                   0
dtype: int64

In [163]:
# stn_asset_water_act.dropna(how='any', inplace=True)
hi


NameError: name 'hi' is not defined

In [149]:
stn_asset_water_act.to_excel('stn_asset_water_act.xlsx')

In [162]:
water_merged.to_excel('water_merged.xlsx')

In [5]:
import os

path = 'C:\\Users\\Lenovo\\Downloads\\Net Zero 1st Set Data - 30-11-2023 2 (1)\\Business Travel Emission Management\\2023'

count = 0
for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		count += 1
		
print(count)

124


In [40]:
f = 720
s = [*str(f)]
print(sum(list(map(int, s))))

9


In [37]:
s = 'abc'
print(s.split(''))

ValueError: empty separator